# Clima en los cultivos

Uno de los factores principales a considerar durante las cosechas es el clima de la región, está determina las temporadas de cosecha y los tipos de cultivos que pueden existir. Por lo tanto será importante obtener la información metereológica del pais. 

## Extracción de datos

### Consistencia de la nueva información

Pretendemos obtener información del clima del país, sin embargo al observar la información obtenida del SIAP, vemos que la información esta dada según los municipios, por lo tanto debemos procurar que la información climática provenga de cada municipio. 

### Fuente
La información será extraída de 'Historical Weather API', que se puede encontrar en el siguiente elnace: https://open-meteo.com/en/docs/historical-weather-api#latitude=29.0881&longitude=-111.0103&start_date=2020-01-01&hourly=temperature_2m,relativehumidity_2m,precipitation. 

Lo principal de la API, es que proporciona información según las coordenadas geograficas de la región (latitud, longitud). Por lo tanto deberemos extraer las coordendadas de cada municipio.

### Proceso.

Esto lo haremos, mediante la extracción de información del siguiente enlace. 'https://www.coordenadas.com.es/mexico/index.php'. Para ello usaremos algunas librerías.

In [3]:
import pandas as pd
from bs4 import BeautifulSoup
import requests

Entonces, primero analizaremos el enlace principal. 

In [4]:
main_url = 'https://www.coordenadas.com.es/mexico/index.php'
html = requests.get(main_url)
print(html)

<Response [200]>


Vemos que `requests` pudo conectar con la página. Entonces usaremos `BeautifulSoup` para revisar su contenido. 

In [5]:
html_content = html.text
soup = BeautifulSoup(html_content)
data = soup.find_all('a',class_= 'list-group-item')
url_estados = []
estados = []
for link in data:
    lhref = link.get('href')
    ltext = link.text
    url_estados.append(lhref)
    estados.append(ltext)

Entonces a partir del enlace principal, obtenemos los enlaces para obtener las coordenadas de los municipios, considere esté ejemplo con Sonora.

In [9]:
son = estados.index('Sonora')
url_son = url_estados[son]

html_son = pd.read_html(url_son)
Sonora = pd.DataFrame(html_son[0])
Sonora.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 60 entries, 0 to 59
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   Ciudad       60 non-null     object
 1   Coordenadas  60 non-null     object
dtypes: object(2)
memory usage: 1.1+ KB


Obtuvimos las coordenadas de los municipios de Sonora, sin embargo, como se puede apreciar. Sonora tiene 72 municipios, y solo nos presenta 60. Esto es debido a la forma de estructura de la página, como puede observar en la siguiente página. 

In [10]:
print(url_son)

https://www.coordenadas.com.es/mexico/pueblos-de-sonora/26/1


Los municipios de Sonora se encuentran desplegados en varias pestañas, en este caso dos. La página mostrada esta representada por él último número del enlace. Entonces debemos modificarlo, para ello definiremos la siguiente función. 

In [11]:
def modify_tabpage(url,tab):
    url = url[:-1] + str(tab)
    return url

In [91]:
url_base = "https://cuentame.inegi.org.mx/monografias/informacion/"
state = 'son'
url_final = "/territorio/div_municipal.aspx"

url_mun = url_base + state + url_final

mun_html = pd.read_html(url_mun)
municipios = pd.DataFrame(mun_html[3])[['Municipio']]
municipios = municipios.drop(municipios.index[-1])
print(municipios)

s = Sonora['Ciudad']

print(s)


                        Municipio
0                         Aconchi
1                     Agua Prieta
2                          Alamos
3                           Altar
4                        Arivechi
..                            ...
67                Villa Pesqueira
68                         Yécora
69  General Plutarco Elías Calles
70                  Benito Juárez
71         San Ignacio Río Muerto

[72 rows x 1 columns]
0             - Aconchi
1         - Agua-Prieta
2              - Alamos
3               - Altar
4            - Arivechi
            ...        
59         - Trincheras
0            - Tubutama
1                - Ures
2     - Villa-Pesqueira
3              - Yecora
Name: Ciudad, Length: 64, dtype: object


Con esto logramos extraer los datos requeridos

In [ ]:

def get_df_mun(url):
    html = pd.read_html(url)
    df = pd.DataFrame(html[0])
    return df